# 项目目标：
* 挖掘用户流量、用户行为、用户画像以及商品特征。
* 建立指标体系或者业务模型，并实现数据可视化。
* 揪出业务问题并给出一定的优化建议。

In [1]:
# 引入模块
import pandas as pd
import numpy as np
import time as time
from sqlalchemy import create_engine
import pymysql

In [ ]:
# 设置数据处理开始时间
start_time = time.time()

# 设置输入和输出文件路径
input_file = f'.git/UserBehavior/UserBehavior.csv'
output_file = f'.git/UserBehavior/UserBehavior.csv'

# 设置最大读取行数
n_rows = 3000000

# 读取数据并保存到新文件
df = pd.read_csv(input_file, nrows = n_rows)
df.to_csv(output_file, index = False)

# 记录读取数据时间
elapesd_time = time.time() - start_time

print(f'数据处理完成，耗时{elapesd_time:.2f}秒。')
print(f'已保存{len(df):,}行数据至新文件。')

数据处理完成，耗时3.61秒。
已保存3,000,000行数据至新文件。


In [7]:
# 定义数据集新列名
new_column_names = [
    'user_id',
    'product_id',
    'category_id',
    'action_type',
    'event_timestamp'
]

# 读取文件并指定列名
df = pd.read_csv(output_file, names = new_column_names)

# 将时间戳转换为日期时间格式
df['times'] = pd.to_datetime(df['event_timestamp'], unit = 's') + pd.Timedelta(hours = 8)

# 提取日期和时间信息
df['dates'] = df['times'].dt.date
df['hours'] = df['times'].dt.hour
df['weekdays'] = df['times'].dt.day_name()

print('数据基本信息：')
print(df.info())

数据基本信息：
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000001 entries, 0 to 3000000
Data columns (total 9 columns):
 #   Column           Dtype         
---  ------           -----         
 0   user_id          int64         
 1   product_id       int64         
 2   category_id      int64         
 3   action_type      object        
 4   event_timestamp  int64         
 5   times            datetime64[ns]
 6   dates            object        
 7   hours            int32         
 8   weekdays         object        
dtypes: datetime64[ns](1), int32(1), int64(4), object(3)
memory usage: 194.5+ MB
None


In [ ]:
# 查看数据集信息、
rows, columns = df.shape

if rows > 0:
    
    # 查看数据集行数和列名
    print(f'数据包含{rows}行和{columns}列。')
    
    # 查看数据集前5行
    print('\n数据前5行：')
    print(df.head().to_csv(sep = '\t', na_rep = 'nan'))

# 定义日期范围
start_date = ('2017-11-25 00:00:00')
end_date = ('2017-12-03 23:59:59')

# 过滤日期范围内的数据
orginal_rows = len(df)
df = df[(df['times'] >= start_date) & (df['times'] <= end_date)]
filtered_rows = len(df)
filtered_percentage = (filtered_rows / orginal_rows) * 100

print('/n过滤后数据集的基本信息：')
print(df.info())

# 检查缺失值
print('/n缺失值统计：')
print(df.isnull().sum())

# 检查重复值
print('/n重复值统计：')
duplicate_count = df.duplicated().sum()
print(f'处理前重复值数量：{duplicate_count}')
df = df.drop_duplicates()
print(f'处理后重复值数量：{df.duplicated().sum()}')

# 保存处理后的数据集
df.to_csv(r'.git/tbub.csv', index = False)

数据包含3000001行和9列。

数据前5行：
	user_id	product_id	category_id	action_type	event_timestamp	times	dates	hours	weekdays
0	1	2268318	2520377	pv	1511544070	2017-11-25 01:21:10	2017-11-25	1	Saturday
1	1	2333346	2520771	pv	1511561733	2017-11-25 06:15:33	2017-11-25	6	Saturday
2	1	2576651	149192	pv	1511572885	2017-11-25 09:21:25	2017-11-25	9	Saturday
3	1	3830808	4181361	pv	1511593493	2017-11-25 15:04:53	2017-11-25	15	Saturday
4	1	4365585	2520377	pv	1511596146	2017-11-25 15:49:06	2017-11-25	15	Saturday

/n过滤后数据集的基本信息：
<class 'pandas.core.frame.DataFrame'>
Index: 2998534 entries, 0 to 3000000
Data columns (total 9 columns):
 #   Column           Dtype         
---  ------           -----         
 0   user_id          int64         
 1   product_id       int64         
 2   category_id      int64         
 3   action_type      object        
 4   event_timestamp  int64         
 5   times            datetime64[ns]
 6   dates            object        
 7   hours            int32         
 8   weekdays 

In [11]:
df.head()

,user_id,product_id,category_id,action_type,event_timestamp,times,dates,hours,weekdays
0,1,2268318,2520377,pv,1511544070,2017-11-25 01:21:10,2017-11-25,1,Saturday
1,1,2333346,2520771,pv,1511561733,2017-11-25 06:15:33,2017-11-25,6,Saturday
2,1,2576651,149192,pv,1511572885,2017-11-25 09:21:25,2017-11-25,9,Saturday
3,1,3830808,4181361,pv,1511593493,2017-11-25 15:04:53,2017-11-25,15,Saturday
4,1,4365585,2520377,pv,1511596146,2017-11-25 15:49:06,2017-11-25,15,Saturday
